### Baseline
Simple neural netowrk with self-defined logistic regression, linear regression, regularized logistic regression, regularized linear regression, sigmoid function, ReLU function, and softmax function.

### Dataset Analysis:
the dataset includes cat, dog and other species. additionally, an image may contains two species. 
* we should first do face detection from line segments, to segments grouping in order to form a small region or part of the face, to forming an entire face by grouping based on the regions. in the following layer, the model should be able to identify if the face is one of cat, dog, or wild animals. in this part, there will be at least 3 layers given there will be at least one for line segments, at least one for segment grouping into region, amd at least one for face forming based on regions.
* in this case, in certain step we shoudl identify if the picture contains cat, dog, and e.g. rabbit or no. so the here it should be binary classification for each unit(neuron). e.g. only two classes, cat and dog. output y = [ bool_has_cat, bool_has_dog ]. Hence this layer should be for instance using sigmoid as activation function.
* Then, in the following layer, emotion should be identified to learn that y = [cat happy?, dog happy?, cat sad?, dog sad?, cat angry?, dog angry?, cat relax?, dog relax?] which given each emotional class we should be answering yes or no. if the image is without cat, then all emotional classes relating to cat should be zero. if the image contains cat and dog, emotional classes relating cat and dog should at least have two ones such that cat has certain emotoin relating to a emotion category, and dog should also have certain emotion relating to certain category. it is possible that the specie is happy and sad at the same time such that there are more than two ones. It is due to this that for each layer, it should for instance use sigmoid activatoin function.
* given that the chance of having that emotion category is approximated, now this follwoing layer should further learn the level of emotion for each emotional class. for instance, if the image contains cat and dog and that for cat the happy and relaxed are both detected, the two neurons for cat happy and cat relaxed should one compute the level of happy and the other one compute the level of relaxed. the same logic applies to dog. in this case, the function should be ReLU. e.g. y = [level of cat is happy, level of dog is happy, level of cat is sad, level of dog is sad, level of cat is angry, level of dog is angry, level of cat is relaxed, level of dog is relaxed]. if the image contains only cat, then all emtional classes level relating to dog should be zero.
* now this layer should identify based on the level of each emotional class. for instance, let's say, the input to this layer is = [level of cat is happy, level of dog is happy, level of cat is sad, level of dog is sad, level of cat is angry, level of dog is angry, level of cat is relaxed, level of dog is relaxed] and that the image contains both cat and dog. cat is detected as happy and relaxed, dog is detected as happy and sad, the input to this layer is then = [0.5, 0.4, 0.0, 0.5, 0.0, 0.0, 0.7, 0.0], then the output of this layer should contains only two ones being [0, 0, 0, 1, 0, 0, 1, 0] such that the cat is relaxed and dog is sad. this layer hence should use either softmax, or, for each unit, for instance, a sigmoid.

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from keras.callbacks import EarlyStopping

import logging
logging.getLogger('tensorflow').setLevel
tf.autograph.set_verbosity(0)


### 1. Data pre-processing
* Since in the baseline model insteresed implementation should use conventional model, we'll stick with simple neural networks from draft for educational purposes, although it's important to note that this approach will possibily yield a very poor result for such a complex image processing task.
* First, you need to preprocess your images. This involves loading the images, resizing them to a uniform size, converting them to grayscale, and flattening them into vectors.

#### Benefits of Using Grayscale Images
Reduced Complexity: Grayscale images are less complex than color images, making them easier to process with simpler algorithms.
Reduced Computational Load: Grayscale images require less computational power and memory, as they have only one channel compared to three in color images.
Focus on Texture and Shape: Converting to grayscale can help the model focus on the texture and shape information, which might be more relevant for certain tasks like emotion detection in animals.

In [ ]:
# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (48, 48))  # Resize to a fixed size for the model
            images.append(img)
    return images

### 2. Labeling the Data
If one has dataset for training and is without labels, one will need to assign labels to the data. Since we have separate folders for each emotion and labels for each data, we can skipp this part.
#### Define the path to the sub dataset folders

In [ ]:
# Define the path to the dataset folders
happy_folder = "/kaggle/input/pets-facial-expression-dataset/happy"
sad_folder = "/kaggle/input/pets-facial-expression-dataset/Sad"
angry_folder = "/kaggle/input/pets-facial-expression-dataset/Angry"

#### Load data and Combine data

In [ ]:
# Load images and labels for each emotion
happy_images = load_images_from_folder(happy_folder)
sad_images = load_images_from_folder(sad_folder)
angry_images = load_images_from_folder(angry_folder)


# Create labels for each emotion category
happy_labels = [0] * len(happy_images)
sad_labels = [1] * len(sad_images)
angry_labels = [2] * len(angry_images)


# Concatenate images and labels
X = np.array(happy_images + sad_images + angry_images)
y = np.array(happy_labels + sad_labels + angry_labels)

# Normalize pixel values to range [0, 1]
X = X.astype('float32') / 255.0

# One-hot encode the labels
y = np_utils.to_categorical(y, 3)

### 3. Splitting the Data
Split dataset into training and testing sets. This is essential for evaluating the performance of the model.
A common split is 80% for training and 20% for testing.

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 4. Define the Network Architecture
1. Face Detection and Segmentation (line, segments, face)
* **Objective**: Detect and segment animal faces from images.
* **Approach**: This typically requires convolutional neural networks (CNNs) that can identify patterns (like edges, textures) and group them into larger structures (like faces).
* **Layers**: Start with convolutional layers for feature extraction, followed by pooling layers to reduce dimensionality, and fully connected layers for classification.
Activation Functions: ReLU is commonly used in CNNs for its efficiency.
2. Species Identification (if we have the labels)
* **Objective**: Identify whether the image contains a cat, dog, or other species.
* **Approach**: This is a multi-label classification problem (since an image can have more than one label).
* **Layers**: Fully connected layers following the feature extraction layers.
* **Activation Function**:** Sigmoid activation function for each neuron (since it's a binary classification for each species).
3. Final Layer for Emotion Classification
* **Objective**: Classify the predominant emotion for each species.
* **Approach**: This is a classification problem, but with a twist. You're interested in the predominant emotion, which is a bit different from standard classification.
* **Layers**: Fully connected layer.
* **Activation Function**: Softmax if you're classifying one predominant emotion per species, or sigmoid for binary classification of each emotion.

### Additional Considerations:
* **Data Preprocessing**: Ensure images are properly preprocessed (normalized, resized, etc.).
* **Model Complexity**: This is a complex model. Start with a simpler version and iteratively add complexity.
* **Training Data**: You'll need a large and well-labeled dataset for this task, especially for the emotion detection and intensity levels.
* **Evaluation Metrics**: Choose appropriate metrics for each stage (accuracy, F1 score, mean squared error for intensity levels, etc.).
* **Computational Resources**: This model might require significant computational resources, especially for training.

### 5. Configure the NN
Since we have four output classes (happy, sad, angry, relaxed), the last output layer should have 4 neurons with a softmax activation function for multi-class classification.

In [ ]:
# Build the simple neural netowrk model

model = Sequential(
    [               
        Flatten(input_shape=(48, 48)),  # The input shape is the size of the images,    #specify input size
        ### START CODE HERE ### 
        Dense(512, activation="relu", name="line"),
        Dense(256, activation="relu", name="segment"),
        Dense(128, activation="relu", name="face"),
        Dense(3, activation="relu", name="species"),
        Dense(3, activation="softmax", name="emotion"),
        ### END CODE HERE ### 
    ], name = "pet_emotion" 
)

[line, segment, face, species, emotion] = model.layers[1:]
#### Examine Weights shapes
W1,b1 = line.get_weights()
W2,b2 = segment.get_weights()
W3,b3 = face.get_weights()
W4,b4 = species.get_weights()
W5,b5 = emotion.get_weights()

print(f"W1 shape = {W1.shape}, b1 shape = {b1.shape}")
print(f"W2 shape = {W2.shape}, b2 shape = {b2.shape}")
print(f"W3 shape = {W3.shape}, b3 shape = {b3.shape}")
print(f"W4 shape = {W4.shape}, b4 shape = {b4.shape}")
print(f"W5 shape = {W5.shape}, b5 shape = {b5.shape}")
# Compile the model
model.compile(
    loss=CategoricalCrossentropy(from_logits=False),
    optimizer=Adam(learning_rate=0.001),
    metrics=[CategoricalAccuracy()]
)

# # Define the model
# model = Sequential([
#     Flatten(input_shape=(48, 48)),
#     Dense(512, activation='relu'),
#     Dense(256, activation='relu'),
#     Dense(128, activation='relu'),
#     Dense(3, activation='softmax')  # Output layer for 3 classes with softmax activation
# ])

# # Compile the model
# model.compile(
#     loss=CategoricalCrossentropy(from_logits=False),
#     optimizer=Adam(learning_rate=0.001),
#     metrics=[CategoricalAccuracy()]
# )

model.save('facial_expression_model.h5')

We can examine details of the model by first extracting the layers with `model.layers` and then extracting the weights with `layerx.get_weights()` as shown below.

### 6. Train the NN

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, validation_split=0.1)  # 48 to 0.1874, 100 to 0.05

# For training accuracy and loss
training_accuracy = history.history['categorical_accuracy']
training_loss = history.history['loss']

# For validation accuracy and loss
validation_accuracy = history.history['val_categorical_accuracy']
validation_loss = history.history['val_loss']

### 7. Evaluate

In [ ]:
# # Predict on test data
import matplotlib.pyplot as plt

epochs = range(1, 101)  # number of epochs = 1000

# Plot training and validation accuracy
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, training_accuracy, 'b-', label='Training accuracy')
plt.plot(epochs, validation_accuracy, 'r-', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs, training_loss, 'b-', label='Training loss')
plt.plot(epochs, validation_loss, 'r-', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


#### Visualizing Test Angry

In [ ]:
from keras.models import load_model
# Load the saved model
loaded_model = load_model("facial_expression_model.h5")



# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (48, 48))  # Resize to a fixed size for the model
            images.append(img)
    return images


# Load a custom test image
custom_test_image_path = "/kaggle/input/pets-facial-expression-dataset/Angry/35.jpg"

custom_test_image = cv2.imread(custom_test_image_path)
custom_test_image = cv2.cvtColor(custom_test_image, cv2.COLOR_BGR2GRAY)
custom_test_image = cv2.resize(custom_test_image, (48, 48))
custom_test_image = custom_test_image.astype('float32') / 255.0

# Reshape the image to match the model input shape
custom_test_image = np.expand_dims(custom_test_image, axis=0)
custom_test_image = np.expand_dims(custom_test_image, axis=-1)

# Make predictions on the custom test image
prediction = loaded_model.predict(custom_test_image)
prediction_prob = prediction[0]

emotion_label = np.argmax(prediction[0])

# Map the predicted label to emotion class
emotion_classes = {0: 'happy', 1: 'sad', 2: 'angry'}
predicted_emotion = emotion_classes[emotion_label]

# Print the custom test image and its predicted label
print(f"Predicted Emotion: {predicted_emotion}")
print(f"Confidence [happy, sad, angry]: {prediction_prob}")

import matplotlib.pyplot as plt

#Display the custom test image using matplotlib
plt.imshow(custom_test_image[0, :, :, 0])
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()

from PIL import Image
# Display the original custom test image using PIL
img_pil = Image.open(custom_test_image_path)
plt.imshow(np.array(img_pil))
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()


#### Visualize Test Happy

In [ ]:
from keras.models import load_model
# Load the saved model
loaded_model = load_model("facial_expression_model.h5")



# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (48, 48))  # Resize to a fixed size for the model
            images.append(img)
    return images


# Load a custom test image
custom_test_image_path = "/kaggle/input/pets-facial-expression-dataset/happy/003.jpg"

custom_test_image = cv2.imread(custom_test_image_path)
custom_test_image = cv2.cvtColor(custom_test_image, cv2.COLOR_BGR2GRAY)
custom_test_image = cv2.resize(custom_test_image, (48, 48))
custom_test_image = custom_test_image.astype('float32') / 255.0

# Reshape the image to match the model input shape
custom_test_image = np.expand_dims(custom_test_image, axis=0)
custom_test_image = np.expand_dims(custom_test_image, axis=-1)

# Make predictions on the custom test image
prediction = loaded_model.predict(custom_test_image)
prediction_prob = prediction[0]

emotion_label = np.argmax(prediction[0])

# Map the predicted label to emotion class
emotion_classes = {0: 'happy', 1: 'sad', 2: 'angry'}
predicted_emotion = emotion_classes[emotion_label]

# Print the custom test image and its predicted label
print(f"Predicted Emotion: {predicted_emotion}")
print(f"Confidence [happy, sad, angry]: {prediction_prob}")

import matplotlib.pyplot as plt

# Display the custom test image using matplotlib
plt.imshow(custom_test_image[0, :, :, 0])
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()

from PIL import Image
# Display the original custom test image using PIL
img_pil = Image.open(custom_test_image_path)
plt.imshow(np.array(img_pil))
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()

#### Visualizing Test Sad

In [ ]:
from keras.models import load_model
# Load the saved model
loaded_model = load_model("facial_expression_model.h5")



# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (48, 48))  # Resize to a fixed size for the model
            images.append(img)
    return images


# Load a custom test image
custom_test_image_path = "/kaggle/input/pets-facial-expression-dataset/Sad/001.jpg"

custom_test_image = cv2.imread(custom_test_image_path)
custom_test_image = cv2.cvtColor(custom_test_image, cv2.COLOR_BGR2GRAY)
custom_test_image = cv2.resize(custom_test_image, (48, 48))
custom_test_image = custom_test_image.astype('float32') / 255.0

# Reshape the image to match the model input shape
custom_test_image = np.expand_dims(custom_test_image, axis=0)
custom_test_image = np.expand_dims(custom_test_image, axis=-1)

# Make predictions on the custom test image
prediction = loaded_model.predict(custom_test_image)
prediction_prob = prediction[0]

emotion_label = np.argmax(prediction[0])

# Map the predicted label to emotion class
emotion_classes = {0: 'happy', 1: 'sad', 2: 'angry'}
predicted_emotion = emotion_classes[emotion_label]

# Print the custom test image and its predicted label
print(f"Predicted Emotion: {predicted_emotion}")
print(f"Confidence [happy, sad, angry]: {prediction_prob}")

import matplotlib.pyplot as plt

#Display the custom test image using matplotlib
plt.imshow(custom_test_image[0, :, :, 0])
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()

from PIL import Image
# Display the original custom test image using PIL
img_pil = Image.open(custom_test_image_path)
plt.imshow(np.array(img_pil))
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()



## Limitations
* Feature Extraction: This approach uses very basic feature extraction (flattening the image), which might not capture the necessary details for accurate emotion classification.
* Model Complexity: Logistic regression is quite basic for image classification tasks.
* Data Quality: The quality and size of your dataset will significantly impact the performance of your model.